In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import countDistinct
sc=SparkContext()

22/11/25 18:47:34 WARN Utils: Your hostname, Tobiass-MacBook.local resolves to a loopback address: 127.0.0.1; using 10.27.90.105 instead (on interface en0)
22/11/25 18:47:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 18:47:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/25 18:47:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [26]:
import numpy as np


Pseude Code:

agrupar por user, device, gt
    para x, y, z sacar
        median
        desv
        max
        min

join data from different datasets into RDDs
Analize RDDs

Don´t repeat code!
avoid many (unnecessary) lines of code / repetitions

In [20]:
#columns = ["index", "arrival_time", "creation_time", "x", "y", "z", "user", "model", "device", "activity"]

pa = sc.textFile('Phones_accelerometer.csv')

pg = sc.textFile('Phones_gyroscope.csv')

wa = sc.textFile('Watch_accelerometer.csv')

wg = sc.textFile('Watch_gyroscope.csv')


In [123]:
split = wg.map(lambda x : x.split(","))
keyed = split.map(lambda x : (x[6] + "," + x[7]+ "," + x[9], np.array([1, float(x[3]), float(x[4]), float(x[5]), float(x[3])**2, float(x[4])**2, float(x[5])**2])))

sum = keyed.reduceByKey(lambda x, y : x + y)
std = sum.map(lambda x : (x[0], np.array([np.sqrt((x[1][4]/x[1][0])-(x[1][1]/x[1][0])**2), np.sqrt((x[1][5]/x[1][0])-(x[1][2]/x[1][0])**2), np.sqrt((x[1][6]/x[1][0])-(x[1][3]/x[1][0])**2)])))
mean = sum.map(lambda x : (x[0], np.array((x[1][1]/x[1][0], x[1][2]/x[1][0], x[1][3]/x[1][0]))))

#sum_std = keyed.map(lambda x: (x[1][1]  - mean[1][1])**2)
'''std = sum_std.reduce(lambda x,y: x + y)
std = np.sqrt(std/heights.count())'''

sum.collect()

[('a,gear,stand',
  array([5047.        ,  111.77420486, -165.88138325, -360.541152  ,
           29.48112279,   21.11171523,   38.46312038])),
 ('a,gear,null',
  array([ 3.43000000e+02,  7.74154314e+00, -1.25568048e+01, -2.45913637e+01,
          2.83401834e-01,  5.05536017e-01,  1.78524658e+00])),
 ('a,gear,sit',
  array([4610.        ,  104.67288548, -190.95770893, -322.57645873,
           17.37226198,   22.50625132,   32.00580686]))]

In [132]:
def read_in(filename):
    #read in file and split at ","
    rdd = sc.textFile(filename)
    split_rdd = rdd.map(lambda x : x.split(","))

    #remove unnecessary variables, create primary key (user + model + gt), create floats: counter variable, x, y, z
    keyed = split_rdd.map(lambda x : (x[6] + "," + x[7]+ "," + x[9], np.array([1, float(x[3]), float(x[4]), float(x[5])])))
    return keyed

In [141]:
def calc_mean(rdd):
    # sum up numericals from input rdd --> x[1]: count per key, sum(x), sum(y), sum(z)
    sum = rdd.reduceByKey(lambda x, y : x + y)
    # divide sums by count
    mean = sum.map(lambda x : (x[0], np.array((x[1][1]/x[1][0], x[1][2]/x[1][0], x[1][3]/x[1][0]))))

    return mean


In [134]:
def calc_std(rdd):
    #remove unnecessary variables, create primary key (user + model + gt), create floats: counter variable, x, y, z, x^2, y^2, z^2 
    work_rdd = rdd.map(lambda x : (x[0], np.array([1, x[1][1], x[1][2], x[1][3], (x[1][1])**2, (x[1][2])**2, (x[1][3])**2])))

    # sum up individual numerical values
    sum = work_rdd.reduceByKey(lambda x, y : x + y)
    # calculate std as follows: sqrt( (sum(x^2)/N) - (sum(x)/N)^2 )
    std = sum.map(lambda x : (x[0], np.array([np.sqrt((x[1][4]/x[1][0])-(x[1][1]/x[1][0])**2), np.sqrt((x[1][5]/x[1][0])-(x[1][2]/x[1][0])**2), np.sqrt((x[1][6]/x[1][0])-(x[1][3]/x[1][0])**2)])))

    return std

In [152]:
def find_max(rdd):
    max = rdd
    return max

In [203]:
out = read_in('Watch_gyroscope.csv')

work = out.map(lambda x: (x[0], x[1][1]))
#out.reduceByKey(max).collect()


In [145]:
def find_min(rdd):
    return rdd

In [135]:
def process(filename):
    rdd = read_in(filename)
    mean = calc_mean(rdd)
    std = calc_std(rdd)

    return mean